In [1]:
from functools import reduce

import folium
import geopandas as gpd
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    RobustScaler,
    StandardScaler,
)
from sklearn.neighbors import KDTree
from xgboost import XGBRegressor
import sys

In [2]:
sys.path.append(".")

In [3]:
from utils import attach_site_context
from utils import calculate_new_site_distances
from utils import build_df_place_to_predict_knn
from utils import poi_counts_within_2km
from utils import predict_new_site_total_volume_knn
from utils import predict_new_site_total_volume_knn_weighted
from utils import estimate_site_investment
from utils import calculate_ROI

In [4]:
import pickle

with open("saved_vars.pkl", "rb") as f:
    data = pickle.load(f)


df_daily = data["df_daily"]
district_coverage = data["district_coverage"]
miscellaneous_costs = data["miscellaneous_costs"]
chargers_market_price = data["chargers_market_price"]
chargers_gdf = data["chargers_gdf"]
gdf_full_updated = data["gdf_full_updated"]
land_price = data["land_price"]
poi_per_district = data["poi_per_district"]
districts_gdf = data["districts_gdf"]
distance_long = data["distance_long"]
unique_sites = data['unique_sites']
place_1 = data['place_1']
place_1_df = data['place_1_df']
place_2 = data['place_2']
place_3 = data['place_3']
place_3_df = data['place_3_df']


In [5]:
lat_min, lat_max = 22.46557, 22.818918
lon_min, lon_max = 113.784724, 114.516991
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

map_szx= folium.Map(location=[center_lat, center_lon], zoom_start=11)

folium.Rectangle(
    bounds=[[lat_min, lon_min], [lat_max, lon_max]],
    color="blue", fill=True, fill_opacity=0.1
).add_to(map_szx)

for _, row in unique_sites.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(map_szx)
for _, row in poi_per_district.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='yellow',
        fill=True,
        fill_opacity=0.6
    ).add_to(map_szx)
map_szx